In [1]:
## Importo librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
## leo los dos archivos
trainlimpio= pd.read_csv('train_con_eda')
test= pd.read_parquet ('test.parquet')

In [3]:
#### Realizo con test, las misma operaciones en train

## Borro las columnas lat y long
borrar= ['lat', 'long']
test.drop(columns= borrar, inplace= True)
## Completo los NaN para el modelo
test['laundry_options'] = test.laundry_options.fillna(test.laundry_options.mode()[0])
test['parking_options'] = test.parking_options.fillna(test.parking_options.mode()[0])
## Borro las columnas que no son para el analisis
Otro_borrar= ['id', 'url', 'region_url', 'image_url', 'description', 'state']
test.drop(columns= Otro_borrar, inplace= True)
## Cambio el tipo de dato
test['baths']=test['baths'].astype (int)

In [4]:
#### Ahora trabajo con las categoricas de test

categoricas= ['type', 'laundry_options', 'parking_options', 'region']
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

#Veo las categoricas
print(test[categoricas].head())

# CreoLabelEncoder : le
le = LabelEncoder()

# Aplico LabelEncoder a las columnas
test[categoricas] = test[categoricas].apply(lambda x: le.fit_transform(x))

# Veo las 5 filas de las categoricas
print(test[categoricas].head())

           type  laundry_options     parking_options                   region
0  manufactured      w/d in unit  off-street parking                ann arbor
1     apartment      w/d hookups  off-street parking                  vermont
2     apartment      w/d in unit             carport                ann arbor
3     apartment      w/d in unit     attached garage  fort collins / north CO
4     apartment  laundry on site  off-street parking          charlottesville
   type  laundry_options  parking_options  region
0     8                4                4       9
1     0                3                4     370
2     0                4                1       9
3     0                4                0     112
4     0                1                4      53


In [5]:
#### ahora escalamos los datos numericos de ambos

from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
scl.fit(trainlimpio[['sqfeet']]) 

trainlimpio['sqfeet'] = scl.transform(trainlimpio[['sqfeet']]) 


from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
scl.fit(test[['sqfeet']]) 

test['sqfeet'] = scl.transform(test[['sqfeet']]) 

#### Comienzo con el modelo: para un mejor análisis, considero trabajar con todas las columnas

In [6]:
X= trainlimpio[['type', 'sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed','wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'laundry_options', 'parking_options', 'region']]
y= trainlimpio[['category_price']]

In [7]:
# Separamos los datos
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [8]:
# Importamos la clase que contiene el algoritmo de regresíon logística

from sklearn.linear_model import LogisticRegression

# Instanciamos el modelo y lo entrenamos

clf = LogisticRegression()

# Lo entrenamos

clf.fit(X_train, y_train)

C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [9]:
## Declaro mi x_test que tiene las mismas columnas que el train
X_test= test[['type', 'sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed','wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'laundry_options', 'parking_options', 'region']]

In [10]:
# Predecimos con el set de testeo

y_pred = clf.predict(X_test)

In [12]:
## Obtengo un dataframe
y_predict = pd.DataFrame(y_pred, columns=['pred'])

In [13]:
y_predict

,pred
0,0
1,0
2,0
3,0
4,1
...,...
38493,0
38494,0
38495,0
38496,0


In [14]:
## Exporto el resultado
y_predict.to_csv('SoleGon4.csv', index=False)